In [134]:
import pandas as pd
import os

In [135]:
def removeUnamed(df):
    columnList = df.columns.tolist()
    newColumnList = columnList.copy()
    i = 0
    while i<len(newColumnList):
        if "Unnamed" in newColumnList[i]:
            stats = newColumnList[i-1].split(" / ")
            newColumnList[i-1] = stats[0]
            newColumnList[i] = " ".join(stats)
        i+=1
    columnsRenamed = dict(zip(columnList,newColumnList))
    df = df.rename(columns=columnsRenamed)
    return df

In [136]:
files = os.listdir("UCSD")

In [137]:
playersData = pd.read_excel("UCSD/"+files[0])
playersData = removeUnamed(playersData)
playersData = playersData.assign(Player = files[0].split(".")[0])
for file in files:
    df = pd.read_excel('UCSD/'+file)
    df = removeUnamed(df)
    df = df.assign(Player = file.split(".")[0])
    playersData = pd.concat([playersData,df])
playersData = playersData[playersData.get("Match").apply(lambda match: "UC San Diego" in match)]
playersData = playersData[playersData.get("Date").apply(lambda date: '2023' in date)]

In [138]:
playerPosition = {"Wilson":"GK","Shor":"GK","Wellerstein":"DEF","McDonnell":"DEF","Walker":"DEF","Gibson":"DEF","Valverde":"MID","Redington":"MID","Kawamura":"MID","Allen":"MID","Iribarren":"MID","Jacobus":"MID","Place":"MID","Hagan":"MID","McGee":"MID","Lin":"MID","Kim":"MID","Carvalho":"ATT","Arens":"ATT","Foltyn":"ATT"}
playersData =playersData.assign(Position = playersData.get("Player").apply(lambda player: playerPosition[player]))

In [139]:
gameData = pd.read_csv("WholeGameData.csv")
gameData = gameData[gameData.get("Team") == "UC San Diego Tritons"].drop("Team",axis = 1)

In [140]:
seasonSum = playersData.drop(["Match","Competition","Date","Position"],axis = 1).groupby("Player").sum().reset_index()

In [141]:
dfs = [playersData, seasonSum]
for df in dfs:
    df.insert(6,"Shot accuracy",df.get("Shots on target")/df.get("Shots"))
    df.insert(6,"Pass accuracy",df.get("Passes accurate")/df.get("Passes"))
    df.insert(6,"Long pass accuracy",df.get("Long passes accurate")/df.get("Long passes"))
    df.insert(6,"Crosses accuracy",df.get("Crosses accurate")/df.get("Crosses"))
    df.insert(6,"Dribbles accuracy",df.get("Dribbles successful")/df.get("Dribbles"))
    df.insert(6,"Duels accuracy",df.get("Duels won")/df.get("Duels"))
    df.insert(6,"Aerial duels accuracy",df.get("Aerial duels won")/df.get("Aerial duels"))
    df.insert(6,"Defensive duels accuracy",df.get("Defensive duels won")/df.get("Defensive duels"))
    df.insert(6,"Loose ball duels accuracy",df.get("Loose ball duels won")/df.get("Loose ball duels"))
    df.insert(6,"Sliding tackles accuracy",df.get("Sliding tackles successful")/df.get("Sliding tackles"))
    df.insert(6,"Offensive duels accuracy",df.get("Offensive duels won")/df.get("Offensive duels"))
    df.insert(6,"Through pass accuracy",df.get("Through passes accurate")/df.get("Through passes"))
    df.insert(6,"Passes to final third accuracy",df.get("Passes to final third accurate")/df.get("Passes to final third"))
    df.insert(6,"Passes to penalty area accuracy",df.get("Passes to penalty area accurate")/df.get("Passes to penalty area"))
    df.insert(6,"Forward passes accuracy",df.get("Forward passes accurate")/df.get("Forward passes"))
    df.insert(6,"Back passes accuracy",df.get("Back passes accurate")/df.get("Back passes"))
    df.insert(6, "Forward passes percentage",df.get("Forward passes")/df.get("Passes") )
    df.insert(6, "Back passes percentage",df.get("Back passes")/df.get("Passes") )
    df.insert(6, "Passes to final third percentage",df.get("Passes to final third")/df.get("Passes") )
    df.insert(6, "Long passes percentage",df.get("Long passes")/df.get("Passes") )


In [142]:
seasonSum =seasonSum.assign(Position = seasonSum.get("Player").apply(lambda player: playerPosition[player]))
seasonSum = seasonSum.set_index("Player")

In [143]:
playerTeamStats = ['Shots', 'Shots on target', 'xG', 'Passes','Passes accurate', 'Long passes', 'Long passes accurate', 'Crosses','Crosses accurate','Duels',
                   'Duels won', 'Aerial duels', 'Aerial duels won', 'Interceptions','Losses','Recoveries','Defensive duels', 'Defensive duels won','Offensive duels', 
                   'Offensive duels won','Touches in penalty area', 'Offsides','Passes to final third','Passes to final third accurate','Forward passes','Forward passes accurate', 'Back passes', 'Back passes accurate']
    
for stat in playerTeamStats:
    statsCol = []
    for rowNum in range(playersData.shape[0]):
        row = playersData.iloc[rowNum]
        teamStat = gameData[gameData.get("Match")==row.get("Match")].get(stat).iloc[0]
        playerStat = row.get(stat)
        statsCol.append(playerStat/teamStat)
    playersData.insert(6,stat+" percentage of team",statsCol)
first = ["Player"]+playersData.columns.tolist()[:5]
second = sorted(playersData.columns.tolist()[5:-1])
playersData = playersData.get(first+second)

C:\Users\zcoch\AppData\Local\Temp\ipykernel_12280\1441350896.py:11: RuntimeWarning: invalid value encountered in scalar divide
  statsCol.append(playerStat/teamStat)
C:\Users\zcoch\AppData\Local\Temp\ipykernel_12280\1441350896.py:11: RuntimeWarning: invalid value encountered in scalar divide
  statsCol.append(playerStat/teamStat)


In [144]:
seasonSum

,Minutes played,Total actions,Total actions successful,Goals,Assists,Long passes percentage,Passes to final third percentage,Back passes percentage,Forward passes percentage,Back passes accuracy,...,Shots against,Saves,Saves with reflexes,Exits,Passes to GK,Passes to GK accurate,Goal kicks,Short goal kicks,Long goal kicks,Position
Player,,,,,,,,,,,,,,,,,,,,,
Allen,1894,1592,926,8,4,0.055409,0.150396,0.189974,0.234828,0.875000,...,0,0,0,0,6,6,0,0,0,MID
Arens,465,354,137,0,0,0.000000,0.047619,0.369048,0.166667,0.806452,...,0,0,0,0,1,1,0,0,0,ATT
Carvalho,1108,811,367,3,3,0.052209,0.092369,0.277108,0.136546,0.869565,...,0,0,0,0,0,0,0,0,0,ATT
Gibson,975,542,329,0,0,0.100313,0.200627,0.125392,0.548589,0.850000,...,0,0,0,0,3,3,0,0,0,DEF
Hagan,384,231,101,0,1,0.046512,0.139535,0.279070,0.197674,0.916667,...,0,0,0,0,0,0,0,0,0,MID
Iribarren,1374,893,515,9,0,0.096203,0.182278,0.136709,0.334177,0.981481,...,0,0,0,0,4,4,0,0,0,MID
Jacobus,988,605,348,1,1,0.078370,0.141066,0.172414,0.423197,0.818182,...,0,0,0,0,2,2,0,0,0,MID
Kawamura,1466,963,595,0,0,0.102823,0.161290,0.183468,0.340726,0.945055,...,0,0,0,0,3,3,0,0,0,MID
Kim,576,366,156,1,1,0.033898,0.101695,0.228814,0.228814,0.814815,...,0,0,0,0,0,0,0,0,0,MID


In [145]:
first = playersData.columns.tolist()[:6]
second = sorted(playersData.columns.tolist()[6:])
playersData = playersData.get(first+second)
first = ["Position","Minutes played"]
second = sorted(seasonSum.columns.tolist()[1:-1])
seasonSum = seasonSum.get(first+second)

In [146]:
seasonSum.to_csv("SeasonSums.csv",index=True)
playersData.to_csv("PlayerGameDate.csv",index=False)